In [13]:
import pycalphad
from pycalphad.variables import *

conditions = dict()
list_conditions = conditions
database = None
list_database = database
components = None
list_components = components
phases = None
list_phases = None
list_equilibrium = None

def set_conditions(**kwargs):
    global conditions
    for key, value in kwargs.items():
        splitkey = key.split('_')
        if len(splitkey) > 1:
            key = getattr(pycalphad.variables, splitkey[0], None)
            key = key('_'.join(splitkey[1:]))
        else:
            key = getattr(pycalphad.variables, splitkey[0], None)
        conditions[key] = value

def switch_database(filename):
    global database, phases, list_database, list_phases
    database = pycalphad.Database(filename)
    list_database = database
    phases = sorted(database.phases.keys())
    list_phases = phases

def define_components(*args):
    global components, list_components
    components = sorted(pycalphad.variables.Species(x) for x in args)
    list_components = components

def define_phases(*args):
    global phases, list_phases
    phases = sorted(args)
    list_phases = phases

class _executable(type):
    def __repr__(cls):
        cls.execute()
        return 'Operation complete'

class calculate_equilibrium(metaclass=_executable):
    __metaclass__ = _executable
    @staticmethod
    def execute():
        global list_equilibrium
        list_equilibrium = pycalphad.equilibrium(database, components, phases, conditions)

In [14]:
switch_database('NI_AL_DUPIN_2001.TDB')
set_conditions(T=300,P=101325,X_AL=0.3,N=1)
define_components('AL', 'NI', 'VA')
define_phases('LIQUID', 'FCC_A1')
calculate_equilibrium

Operation complete

In [15]:
list_equilibrium

<xarray.Dataset>
Dimensions:    (N: 1, P: 1, T: 1, X_AL: 1, component: 2, internal_dof: 2, vertex: 3)
Coordinates:
  * N          (N) float64 1.0
  * P          (P) float64 1.013e+05
  * T          (T) float64 300.0
  * X_AL       (X_AL) float64 0.3
  * vertex     (vertex) int32 0 1 2
  * component  (component) <U2 'AL' 'NI'
Dimensions without coordinates: internal_dof
Data variables:
    NP         (N, P, T, X_AL, vertex) float64 1.0 nan nan
    GM         (N, P, T, X_AL) float64 -3.589e+04
    MU         (N, P, T, X_AL, component) float64 -1.1e+05 -4.138e+03
    X          (N, P, T, X_AL, vertex, component) float64 0.3 0.7 ... nan nan
    Y          (N, P, T, X_AL, vertex, internal_dof) float64 0.3 0.7 ... nan nan
    Phase      (N, P, T, X_AL, vertex) <U6 'LIQUID' '' ''
Attributes:
    engine:   pycalphad 0.7.1.post2+2.g6b5e9058
    created:  2019-03-07T16:16:32.533941

In [17]:
list_conditions, list_phases, list_components

({T: 300, P: 101325, X_AL: 0.3, N: 1},
 ['FCC_A1', 'LIQUID'],
 [Species('AL', 'AL1.0'), Species('NI', 'NI1.0'), Species('VA', 'VA1.0')])